In [1]:
# -*- coding: utf-8 -*-
"""
Generated by ArcGIS ModelBuilder on : 2024-01-22 16:18:29
"""
import arcpy
from arcpy.ia import *
from sys import argv

In [2]:
GDB_python = r'D:\\Mali_FY24_D\\Mali_Health_FY24_python.gdb\\'
GDB_temp = r'D:\\Mali_FY24_D\\NF_scratch.gdb\\'
GDB_ACA = r'D:\\Mali_FY24_D\\NF_ACA1.gdb\\'
GDB_RCA = r'R:\\MLI\\GEO\\Team\\Projects\\MLI_Health_FY24_R\\NF_RCA1.gdb\\'
print('\n'.join([GDB_python, GDB_temp, GDB_ACA, GDB_RCA]))

D:\\Mali_FY24_D\\Mali_Health_FY24_python.gdb\\
D:\\Mali_FY24_D\\NF_scratch.gdb\\
D:\\Mali_FY24_D\\NF_ACA1.gdb\\
R:\\MLI\\GEO\\Team\\Projects\\MLI_Health_FY24_R\\NF_RCA1.gdb\\


In [3]:
pG20j1_tb = GDB_temp + "pG20j1_tb"
checking = arcpy.SearchCursor(pG20j1_tb)

for row in checking:
    x = row.getValue("SUM")

print(x)

8861.701286077508


In [14]:
raster = 'NF_50_ACA'
print("NF_code = " + "'{}'".format(raster))
print("'NF_code' = " + '"NF_50_ACA"')

NF_code = 'NF_50_ACA'
'NF_code' = "NF_50_ACA"


In [15]:
raster = 'NF_50_ACA'
whereClause = "NF_code = " + "{}".format(raster)

with arcpy.da.UpdateCursor(GDB_python + "Eligible_NF_collect", # Path to table
                           ["NF_code", "testRow"], # Fields of interest. Referred to in loop by their index in this list
                           where_clause = "NF_code = " + "'{}'".format(raster)) as cursor: # Fields of interest
        
    for row in cursor:
        # Update the value in the second field using the results from the zStat table
        row[1] = 999
        cursor.updateRow(row)

In [3]:
def PeopleGain_NthNewFacility(RCA_Rasters = GDB_RCA[:-2], ACA_masks = GDB_ACA[:-2]):

    # To allow overwriting outputs change overwriteOutput option to True.
    arcpy.env.overwriteOutput = True

    # Check out any necessary licenses.
    arcpy.CheckOutExtension("spatial")
    arcpy.CheckOutExtension("ImageAnalyst")
    arcpy.CheckOutExtension("ImageExt")
    
    # Inputs
    Eligible_NF = GDB_python + "Eligible_NF"
    ADM1_Dissolve = GDB_python + "ADM1_Dissolve"
    wpop = arcpy.Raster(GDB_python + "wpop")
    pxHFj_RCA_Full = arcpy.Raster(GDB_python + "pxHFj1_RCA_Full")
    COMREF_MXmax = arcpy.Raster(r'C:\Users\wb527163\GIS\Mali_Cdrive\Mali_Health_FY24\Mali_Health_FY24_C\Mali_Health_FY24_C.gdb\HFj1_MXmax')
    R1_MXc360 = arcpy.Raster(GDB_python + "R1_MXc360")
    
    # Intermediate outputs
    pNF_ACA_tb = GDB_temp + "pNF_ACA_tb"
    pxHFj1_RCA_tb = GDB_temp + "pxHFj2_RCA_tb"
    pxHFj_RCA_tb = GDB_temp + "pxHFR1_RCA_tb"
    pG20j1_tb = GDB_temp + "pG20j2_tb"
    
    # Final storage of values
#     NF2 = Eligible_NF #+ "_2"
    Collect = GDB_python + "Eligible_NF_collect"
    
    # Find the list of NFs that will need updated values for the Nth best facility iteration.
    global Overlapping
    Overlapping = []
    
    
    print('Iterating through: ', rasters, '\n')

    for raster in rasters: 
        
        ACA_mask = GDB_ACA + raster
        RCA_selection = GDB_RCA + raster
        expr = '"' + "{}".format(raster) + '"'
        print('Starting: ', expr) #print('\n'.join([ACA_mask, RCA_selection, expr]))
        
        
# --------- Determine whether Nth calculation would change from N-1. ----------
        
        try:
            arcpy.GetRasterProperties_management(arcpy.sa.ExtractByMask(ACA_mask, R1_MXc360), 
                                                 "MAXIMUM").getOutput(0)
            
        # If we are unable to get a max value from the masked object, it is because the two catchments do not overlap.
        except:
            print("Not overlapping. %s" % (raster))
        else:
            print("Overlapping. %s" % (raster))
            Overlapping.append(raster)
    
   
   # for raster in Overlapping:         
            
# --------- pxHFj1_RCA: People-minutes with introduction of New Facility ---------

            print('\n\n---Running People-minutes with an Nth New Facility for %s. %s---' % (raster, time.ctime()))
    
            MX_HFjn = Con(IsNull(Raster(RCA_selection)), # If RCA catchment doesn't cover the area, ...
                          Raster(COMREF_MXmax),          # then multiply by HFj+1 travel time.
                          Con(Raster(RCA_selection) < Raster(COMREF_MXmax), # Otherwise, multiply by whichever is smaller.
                              Raster(RCA_selection),
                              Raster(COMREF_MXmax)))
            
            multiplied = Con(~IsNull(Raster(ACA_mask)), # If the raster cell is within the catchment area, ...
                             Raster(wpop) * MX_HFjn     # then perform calculation.
                            ) # Otherwise, NoData. (No third parameter in Con() defaults to Null.)
            print("Raster calculator finished. %s" % time.ctime())

            with arcpy.EnvManager(parallelProcessingFactor="80%"):
                arcpy.ia.ZonalStatisticsAsTable(in_zone_data=ADM1_Dissolve, zone_field="OBJECTID", 
                                                in_value_raster=multiplied, out_table=pxHFj1_RCA_tb, 
                                                ignore_nodata="DATA", statistics_type="SUM", 
                                                process_as_multidimensional="CURRENT_SLICE", percentile_values=90, 
                                                percentile_interpolation_type="AUTO_DETECT")
            print("Zonal stats finished. %s" % time.ctime())

#             arcpy.management.CalculateField(in_table=pxHFj1_RCA_tb, field="NF_code", expression=expr, 
#                                             expression_type="PYTHON3", field_type="TEXT")
               
            cursor = arcpy.SearchCursor(pxHFj1_RCA_tb) # Table has only one row. No need to specify a where clause.
            for row in cursor:
                pullValue = row.getValue("SUM") # Save the zonal stats result as an object
                
            with arcpy.da.UpdateCursor(Collect, 
                                       ["NF_code", "pxHFj2_RCA"],
                                       where_clause = "NF_code = " + "'{}'".format(raster)) as cursor: # Find the row dedicated to this raster.
                for row in cursor:
                    # Update the value in the parent table to the zStats result
                    row[1] = x
                    cursor.updateRow(row)
            print("Updated Eligible_NF_collect results table. %s" time.ctime())

#             arcpy.management.JoinField(NF2, "NF_code", pxHFj1_RCA_tb, "NF_code", "SUM")
#             print("Joined to copy of New Facilities. %s" % time.ctime())

#             arcpy.management.CalculateField(in_table=NF2, field="pxHFj2_RCA", 
#                                             expression="!SUM! if !SUM! is not None else !pxHFj2_RCA!", 
#                                             expression_type="PYTHON3")
#             print("Transferred to permanent attribute table. %s" % time.ctime())

#             arcpy.management.DeleteField(in_table=NF2, drop_field=["SUM"], method="DELETE_FIELDS")
#             print("Deleted temp field. %s" % time.ctime())

            
            
# --------- pxHFj_RCA: People-minutes at baseline ---------

            print('\n\n---Running People-minutes at baseline for %s. %s---' % (raster, time.ctime()))
            
            with arcpy.EnvManager(cellSize=pxHFj_RCA_Full):
                pxHFj_mask = arcpy.sa.ExtractByMask(pxHFj_RCA_Full, ACA_mask)#; 
            print("Raster calculator finished. %s" % time.ctime())

            with arcpy.EnvManager(parallelProcessingFactor="80%"):
                arcpy.ia.ZonalStatisticsAsTable(in_zone_data=ADM1_Dissolve, zone_field="OBJECTID", 
                                                in_value_raster=pxHFj_mask, out_table=pxHFj_RCA_tb, 
                                                ignore_nodata="DATA", statistics_type="SUM", 
                                                process_as_multidimensional="CURRENT_SLICE", percentile_values=90, 
                                                percentile_interpolation_type="AUTO_DETECT")
            print("Zonal stats finished. %s" % time.ctime())

            
            cursor = arcpy.SearchCursor(pxHFj_RCA_tb) # Table has only one row. No need to specify a where clause.
            for row in cursor:
                pullValue = row.getValue("SUM") # Save the zonal stats result as an object
                
            with arcpy.da.UpdateCursor(Collect, 
                                       ["NF_code", "pxHFR1_RCA"],
                                       where_clause = "NF_code = " + "'{}'".format(raster)) as cursor: # Find the row dedicated to this raster.
                for row in cursor:
                    # Update the value in the parent table to the zStats result
                    row[1] = x
                    cursor.updateRow(row)
            print("Updated Eligible_NF_collect results table. %s" time.ctime())
            
#             arcpy.management.CalculateField(in_table=pxHFj_RCA_tb, field="NF_code", expression=expr, 
#                                             expression_type="PYTHON3", field_type="TEXT")
            
#             arcpy.management.JoinField(NF2, "NF_code", pxHFj_RCA_tb, "NF_code", "SUM")
#             print("Joined to copy of New Facilities. %s" % time.ctime())

#             arcpy.management.CalculateField(in_table=NF2, field="pxHFR1_RCA", 
#                                             expression="!SUM! if !SUM! is not None else !pxHFR1_RCA!", 
#                                             expression_type="PYTHON3")
#             print("Transferred to permanent attribute table. %s" % time.ctime())

#             arcpy.management.DeleteField(in_table=NF2, drop_field=["SUM"], method="DELETE_FIELDS")
#             print("Deleted temp field. %s" % time.ctime())
      
    
            
# --------- pG20j1: People with at least 20% reduction in travel time from New Facility ---------

            print('\n\n---Running People whose travel time is reduced for %s. %s---' % (raster, time.ctime()))

            with arcpy.EnvManager(mask= ACA_mask):
                reduction = Con( ((COMREF_MXmax - MX_HFjn)/COMREF_MXmax) >= 0.2, # If cell has 20% reduction or more, ...
                                wpop                                             # Then keep the population value.
                               )                                                 # Otherwise, change to NoData.
            print("Raster calculator finished. %s" % time.ctime())
            
            with arcpy.EnvManager(parallelProcessingFactor="80%"):
                arcpy.ia.ZonalStatisticsAsTable(in_zone_data=ADM1_Dissolve, zone_field="OBJECTID", 
                                                in_value_raster=reduction, out_table=pG20j1_tb, 
                                                ignore_nodata="DATA", statistics_type="SUM", 
                                                process_as_multidimensional="CURRENT_SLICE", percentile_values=90, 
                                                percentile_interpolation_type="AUTO_DETECT")
            print("Zonal stats finished. %s" % time.ctime())
            
            cursor = arcpy.SearchCursor(pG20j1_tb) # Table has only one row. No need to specify a where clause.
            for row in cursor:
                pullValue = row.getValue("SUM") # Save the zonal stats result as an object
                
            with arcpy.da.UpdateCursor(Collect, 
                                       ["NF_code", "pG20j2"],
                                       where_clause = "NF_code = " + "'{}'".format(raster)) as cursor: # Find the row dedicated to this raster.
                for row in cursor:
                    # Update the value in the parent table to the zStats result
                    row[1] = x
                    cursor.updateRow(row)
            print("Updated Eligible_NF_collect results table. %s" time.ctime())
            
#             arcpy.management.CalculateField(in_table=pG20j1_tb, field="NF_code", expression=expr, 
#                                             expression_type="PYTHON3", field_type="TEXT")
            
#             arcpy.management.JoinField(NF2, "NF_code", pG20j1_tb, "NF_code", "SUM")
#             print("Joined to copy of New Facilities. %s" % time.ctime())

#             arcpy.management.CalculateField(in_table=NF2, field="pG20j2", 
#                                             expression="!SUM! if !SUM! is not None else !pG20j2!", 
#                                             expression_type="PYTHON3")
#             print("Transferred to permanent attribute table. %s" % time.ctime())

#             arcpy.management.DeleteField(in_table=NF2, drop_field=["SUM"], method="DELETE_FIELDS")
#             print("Deleted temp field. %s" % time.ctime())
            
    print(Overlapping)
        

In [4]:
RCA_Rasters = GDB_RCA[:-2]
arcpy.env.workspace  = RCA_Rasters
rasters = arcpy.ListRasters("*", "")
print(rasters)

['NF_1_ACA', 'NF_2_ACA', 'NF_3_ACA', 'NF_4_ACA', 'NF_5_ACA', 'NF_6_ACA', 'NF_7_ACA', 'NF_8_ACA', 'NF_9_ACA', 'NF_10_ACA', 'NF_11_ACA', 'NF_12_ACA', 'NF_13_ACA', 'NF_14_ACA', 'NF_15_ACA', 'NF_16_ACA', 'NF_17_ACA', 'NF_18_ACA', 'NF_19_ACA', 'NF_20_ACA', 'NF_21_ACA', 'NF_22_ACA', 'NF_23_ACA', 'NF_24_ACA', 'NF_25_ACA', 'NF_26_ACA', 'NF_27_ACA', 'NF_28_ACA', 'NF_29_ACA', 'NF_30_ACA', 'NF_31_ACA', 'NF_32_ACA', 'NF_33_ACA', 'NF_34_ACA', 'NF_35_ACA']


In [5]:
if __name__ == '__main__':
    # Global Environment settings
    with arcpy.EnvManager(scratchWorkspace=GDB_temp[:-2], workspace=GDB_python[:-2]):
        PeopleGain_NthNewFacility(*argv[1:])

Iterating through:  ['NF_1_ACA', 'NF_2_ACA', 'NF_3_ACA', 'NF_4_ACA', 'NF_5_ACA', 'NF_6_ACA', 'NF_7_ACA', 'NF_8_ACA', 'NF_9_ACA', 'NF_10_ACA', 'NF_11_ACA', 'NF_12_ACA', 'NF_13_ACA', 'NF_14_ACA', 'NF_15_ACA', 'NF_16_ACA', 'NF_17_ACA', 'NF_18_ACA', 'NF_19_ACA', 'NF_20_ACA', 'NF_21_ACA', 'NF_22_ACA', 'NF_23_ACA', 'NF_24_ACA', 'NF_25_ACA', 'NF_26_ACA', 'NF_27_ACA', 'NF_28_ACA', 'NF_29_ACA', 'NF_30_ACA', 'NF_31_ACA', 'NF_32_ACA', 'NF_33_ACA', 'NF_34_ACA', 'NF_35_ACA'] 

Starting:  "NF_1_ACA"
Overlapping. NF_1_ACA


---Running People-minutes with an Nth New Facility for NF_1_ACA. Tue Feb 13 09:20:38 2024---
Raster calculator finished. Tue Feb 13 09:20:53 2024
Zonal stats finished. Tue Feb 13 10:29:12 2024
Joined to copy of New Facilities. Tue Feb 13 10:29:14 2024
Transferred to permanent attribute table. Tue Feb 13 10:29:14 2024
Deleted temp field. Tue Feb 13 10:29:15 2024


---Running People-minutes at baseline for NF_1_ACA. Tue Feb 13 10:29:15 2024---
Raster calculator finished. Tue Feb 13 

Joined to copy of New Facilities. Wed Feb 14 04:06:14 2024
Transferred to permanent attribute table. Wed Feb 14 04:06:15 2024
Deleted temp field. Wed Feb 14 04:06:16 2024
Starting:  "NF_9_ACA"
Overlapping. NF_9_ACA


---Running People-minutes with an Nth New Facility for NF_9_ACA. Wed Feb 14 04:06:47 2024---
Raster calculator finished. Wed Feb 14 04:06:55 2024
Zonal stats finished. Wed Feb 14 05:12:35 2024
Joined to copy of New Facilities. Wed Feb 14 05:12:37 2024
Transferred to permanent attribute table. Wed Feb 14 05:12:37 2024
Deleted temp field. Wed Feb 14 05:12:38 2024


---Running People-minutes at baseline for NF_9_ACA. Wed Feb 14 05:12:38 2024---
Raster calculator finished. Wed Feb 14 05:13:39 2024
Zonal stats finished. Wed Feb 14 05:13:48 2024
Joined to copy of New Facilities. Wed Feb 14 05:13:50 2024
Transferred to permanent attribute table. Wed Feb 14 05:13:50 2024
Deleted temp field. Wed Feb 14 05:13:51 2024


---Running People whose travel time is reduced for NF_9_ACA. Wed

ExecuteError: ERROR 010005: Unable to allocate memory.
Failed to execute (ZonalStatisticsAsTable).


In [ ]:
Overlap_1 = Overlapping.copy()

In [ ]:
GDB_ACA = r'D:\\Mali_FY24_D\\NF_ACA39.gdb\\'
GDB_RCA = r'R:\\MLI\\GEO\\Team\\Projects\\MLI_Health_FY24_R\\NF_RCA39.gdb\\'
arcpy.env.workspace  = GDB_ACA[:-2]
rasters = arcpy.ListRasters("*", "")
print(rasters)

In [ ]:
if __name__ == '__main__':
    # Global Environment settings
    with arcpy.EnvManager(scratchWorkspace=GDB_temp[:-2], workspace=GDB_python[:-2]):
        PeopleGain_NthNewFacility(*argv[1:])

In [ ]:
Overlap_39 = Overlapping.copy()

In [ ]:
GDB_ACA = r'D:\\Mali_FY24_D\\NF_ACA62.gdb\\'
GDB_RCA = r'R:\\MLI\\GEO\\Team\\Projects\\MLI_Health_FY24_R\\NF_RCA62.gdb\\'
arcpy.env.workspace  = GDB_ACA[:-2]
rasters = arcpy.ListRasters("*", "")
print(rasters)

In [ ]:
if __name__ == '__main__':
    # Global Environment settings
    with arcpy.EnvManager(scratchWorkspace=GDB_temp[:-2], workspace=GDB_python[:-2]):
        PeopleGain_NthNewFacility(*argv[1:])

In [ ]:
Overlap_62 = Overlapping.copy()

In [ ]:
GDB_ACA = r'R:\\MLI\\GEO\\Team\\Projects\\MLI_Health_FY24_R\\NF_ACA102.gdb\\'
GDB_RCA = r'R:\\MLI\\GEO\\Team\\Projects\\MLI_Health_FY24_R\\NF_RCA102.gdb\\'
arcpy.env.workspace  = GDB_ACA[:-2]
rasters = arcpy.ListRasters("*", "")
print(rasters)

In [ ]:
if __name__ == '__main__':
    # Global Environment settings
    with arcpy.EnvManager(scratchWorkspace=GDB_temp[:-2], workspace=GDB_python[:-2]):
        PeopleGain_NthNewFacility(*argv[1:])

In [ ]:
Overlap_102 = Overlapping.copy()

In [ ]:
GDB_ACA = r'D:\\Mali_FY24_D\\NF_ACA121.gdb\\'
GDB_RCA = r'R:\\MLI\\GEO\\Team\\Projects\\MLI_Health_FY24_R\\NF_RCA121.gdb\\'
arcpy.env.workspace  = GDB_ACA[:-2]
rasters = arcpy.ListRasters("*", "")
print(rasters)

In [ ]:
if __name__ == '__main__':
    # Global Environment settings
    with arcpy.EnvManager(scratchWorkspace=GDB_temp[:-2], workspace=GDB_python[:-2]):
        PeopleGain_NthNewFacility(*argv[1:])

In [ ]:
Overlap_121 = Overlapping.copy()

In [ ]:
GDB_ACA = r'R:\\MLI\\GEO\\Team\\Projects\\MLI_Health_FY24_R\\NF_ACA162.gdb\\'
GDB_RCA = r'R:\\MLI\\GEO\\Team\\Projects\\MLI_Health_FY24_R\\NF_RCA162.gdb\\'
arcpy.env.workspace  = GDB_ACA[:-2]
rasters = arcpy.ListRasters("*", "")
print(rasters)

In [ ]:
if __name__ == '__main__':
    # Global Environment settings
    with arcpy.EnvManager(scratchWorkspace=GDB_temp[:-2], workspace=GDB_python[:-2]):
        PeopleGain_NthNewFacility(*argv[1:])

In [ ]:
Overlap_162 = Overlapping.copy()

In [ ]:
GDB_ACA = r'D:\\Mali_FY24_D\\NF_ACA196.gdb\\'
GDB_RCA = r'R:\\MLI\\GEO\\Team\\Projects\\MLI_Health_FY24_R\\NF_RCA196.gdb\\'
arcpy.env.workspace  = GDB_ACA[:-2]
rasters = arcpy.ListRasters("*", "")
print(rasters)

In [ ]:
if __name__ == '__main__':
    # Global Environment settings
    with arcpy.EnvManager(scratchWorkspace=GDB_temp[:-2], workspace=GDB_python[:-2]):
        PeopleGain_NthNewFacility(*argv[1:])

In [ ]:
Overlap_196 = Overlapping.copy()

In [ ]:
GDB_ACA = r'R:\\MLI\\GEO\\Team\\Projects\\MLI_Health_FY24_R\\NF_ACA251.gdb\\'
GDB_RCA = r'R:\\MLI\\GEO\\Team\\Projects\\MLI_Health_FY24_R\\NF_RCA251.gdb\\'
arcpy.env.workspace  = GDB_ACA[:-2]
rasters = arcpy.ListRasters("*", "")
print(rasters)

In [ ]:
if __name__ == '__main__':
    # Global Environment settings
    with arcpy.EnvManager(scratchWorkspace=GDB_temp[:-2], workspace=GDB_python[:-2]):
        PeopleGain_NthNewFacility(*argv[1:])

In [ ]:
Overlap_251 = Overlapping.copy()

In [ ]:
GDB_ACA = r'D:\\Mali_FY24_D\\NF_ACA264.gdb\\'
GDB_RCA = r'R:\\MLI\\GEO\\Team\\Projects\\MLI_Health_FY24_R\\NF_RCA264.gdb\\'
arcpy.env.workspace  = GDB_ACA[:-2]
rasters = arcpy.ListRasters("*", "")
print(rasters)

In [ ]:
if __name__ == '__main__':
    # Global Environment settings
    with arcpy.EnvManager(scratchWorkspace=GDB_temp[:-2], workspace=GDB_python[:-2]):
        PeopleGain_NthNewFacility(*argv[1:])

In [ ]:
Overlap_264 = Overlapping.copy()